In [1]:
import numpy as np
import matplotlib.pyplot as plt
import h5py
import random
import sklearn
import collections
from sklearn.model_selection import train_test_split
import json
import pylab
import tensorflow as tf
import tensorflow.math as tfmath
import tensorflow.keras as keras
from scipy.optimize import curve_fit
from tensorflow.keras import layers, Model
import tensorflow.keras.backend as K
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.models import load_model
from sklearn.metrics import roc_curve, auc
import sklearn.metrics as sk
import qkeras
from qkeras.qlayers import QDense, QActivation
from qkeras.quantizers import quantized_bits, quantized_relu

2024-03-13 23:14:34.725931: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-03-13 23:14:37.569863: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
f=h5py.File('/eos/home-w/wsherman/AD_Work/n_tuples/40MHZ_data/background_for_training.h5','r')
Dataset=np.array(f["Particles"])

#for i, batch in enumerate(Dataset):
#  pt_sum=0
#  for j, particle in enumerate(Dataset[i,:,:]):
#    if particle[3]!=0:
#      pt_sum+=particle[0]
#  for j, particle in enumerate(Dataset[i,:,:]):
#    particle[0]=particle[0]/pt_sum
    
    
Data_Train=Dataset[0:2000000,:,0:3]
Data_Test=Dataset[2000001:3600000,:,0:3]
Data_Validate=Dataset[3600001:4000000,:,0:3]

Data_Train_Flat=np.reshape(Data_Train,(-1,57))
Data_Val_Flat=np.reshape(Data_Validate,(-1,57))

In [3]:
class Sampling(keras.layers.Layer):
    """Uses (z_mean, z_log_var) to sample z, the vector encoding a digit."""

    def call(self, inputs):
        z_mean, z_log_var = inputs
        batch = tf.shape(z_mean)[0]
        dim = tf.shape(z_mean)[1]
        epsilon = tf.keras.backend.random_normal(shape=(batch, dim))
        return z_mean + tf.exp(0.5 * z_log_var) * epsilon


def make_Qencoder(input_dim,h_dim_1,h_dim_2,latent_dim):
    inputs=keras.Input(shape=(input_dim))

    x=QDense(h_dim_1,kernel_quantizer=quantized_bits(16),bias_quantizer=quantized_bits(16))(inputs)
    x=QActivation("quantized_relu(16)")(x)
    x=QDense(h_dim_2,kernel_quantizer=quantized_bits(16),bias_quantizer=quantized_bits(16))(x)
    x=QActivation("quantized_relu(16)")(x)
    z_mean=QDense(latent_dim, kernel_quantizer=quantized_bits(16),bias_quantizer=quantized_bits(16))(x)
    z_logvar=QDense(latent_dim, kernel_quantizer=quantized_bits(16),bias_quantizer=quantized_bits(16))(x)
    z=Sampling()([z_mean,z_logvar])
    encoder=keras.Model(inputs,[z_mean,z_logvar,z],name='encoder')
    return encoder




def make_Qdecoder(input_dim,h_dim_1,h_dim_2,latent_dim):
    inputs=keras.Input(shape=(latent_dim))
    x=QDense(h_dim_2, kernel_quantizer=quantized_bits(16),bias_quantizer=quantized_bits(16))(inputs)
    x=QActivation("quantized_relu(16)")(x)
    x=QDense(h_dim_1,kernel_quantizer=quantized_bits(16),bias_quantizer=quantized_bits(16))(x)
    x=QActivation("quantized_relu(16)")(x)
    z=QDense(input_dim,kernel_quantizer=quantized_bits(16),bias_quantizer=quantized_bits(16))(x)
    decoder=keras.Model(inputs,z,name='decoder')
    return decoder


class QVAE_Model(keras.Model):
    def __init__(self,encoder,decoder,**kwargs):
        super().__init__(**kwargs)
        self.encoder = encoder
        self.decoder = decoder
        self.total_loss_tracker = keras.metrics.Mean(name="total_loss")
        self.reconstruction_loss_tracker = keras.metrics.Mean(
            name="reconstruction_loss"
        )
        self.kl_loss_tracker = keras.metrics.Mean(name="kl_loss")
        self.beta=1

    @property
    def metrics(self):
        return [
            self.total_loss_tracker,
            self.reconstruction_loss_tracker,
            self.kl_loss_tracker,
        ]
    
    def set_beta(self,beta):
        self.beta=beta
    
    def train_step(self, data):
        with tf.GradientTape() as tape:
            z_mean, z_log_var, z = self.encoder(data)
            reconstruction = self.decoder(z)
            #making a masked loss function
            mask = K.cast(K.not_equal(data, 0), K.floatx())
            
            reconstruction_loss = tf.reduce_mean(tf.reduce_sum(keras.losses.mse(mask*data, mask*reconstruction)))
            reconstruction_loss *=(1-self.beta)

            kl_loss = -0.5 * (1 + z_log_var - tf.square(z_mean) - tf.exp(z_log_var))
            kl_loss = tf.reduce_mean(kl_loss)
            kl_loss *=self.beta


            total_loss = reconstruction_loss + kl_loss
            
        grads = tape.gradient(total_loss, self.trainable_weights)
        self.optimizer.apply_gradients(zip(grads, self.trainable_weights))
        self.total_loss_tracker.update_state(total_loss)
        self.reconstruction_loss_tracker.update_state(reconstruction_loss)
        self.kl_loss_tracker.update_state(kl_loss)
        return {
            "loss": self.total_loss_tracker.result(),
            "reco_loss": self.reconstruction_loss_tracker.result(),
            "kl_loss": self.kl_loss_tracker.result(),
        }

    def test_step(self, data):
        z_mean, z_log_var, z = self.encoder(data)
        
        reconstruction = self.decoder(z)
        mask = K.cast(K.not_equal(data, 0), K.floatx())
        reconstruction_loss = tf.reduce_mean(tf.reduce_sum(keras.losses.mse(mask*data, mask*reconstruction)))
        
        reconstruction_loss*=(1-self.beta)

        kl_loss = -0.5 * (1 + z_log_var - tf.square(z_mean) - tf.exp(z_log_var))
        
        #KL loss changed abck to sum as in paper
        kl_loss = tf.reduce_sum(kl_loss)
        
        kl_loss *=self.beta
        
        total_loss = reconstruction_loss + kl_loss
        return {
            "loss": total_loss,
            "reco_loss": reconstruction_loss,
            "kl_loss": kl_loss,
        }

    def call(self, data):
        z_mean,z_log_var,x = self.encoder(data)
        reconstruction = self.decoder(x)
        return {
            "z_mean": z_mean,
            "z_log_var": z_log_var,
            "reconstruction": reconstruction
        }

        
    

In [4]:
#Here is a normalized model with (1-beta)rl beta*Kl loss
beta=0.83
vae_enc=make_Qencoder(57,32,16,3)
vae_dec=make_Qdecoder(57,32,16,3)
vae_40MHZ=QVAE_Model(vae_enc,vae_dec)
vae_40MHZ.set_beta(beta)
opt=keras.optimizers.Adam(learning_rate=0.001)
vae_40MHZ.compile(optimizer=opt)


STOP_PATIENCE = 8
LR_PATIENCE = 4

early_stopping = EarlyStopping(patience=STOP_PATIENCE, restore_best_weights=True)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=LR_PATIENCE, verbose=1)

callbacks = [early_stopping, reduce_lr]

NUM_EPOCHS = 100
BATCH_SIZE = 100

history = vae_40MHZ.fit(x=Data_Train_Flat, validation_split=0.1,epochs=NUM_EPOCHS, batch_size=BATCH_SIZE, callbacks=callbacks,shuffle=True)
vae_40MHZ.save_weights(filepath='/eos/home-w/wsherman/AD_Work/ML_git_repo/AD_trigger_training/trained_models/Different_VAE_Models/Q_keras_16_bit_non_normed_new_beta_{}_v4_no_batchnorm/'.format(beta),save_format='tf')

Epoch 1/100
18000/18000 [==============================] - 74s 4ms/step - loss: 638.8617 - reco_loss: 638.4465 - kl_loss: 0.1394 - val_loss: 415.0768 - val_reco_loss: 415.0768 - val_kl_loss: 0.0000e+00 - lr: 0.0010
Epoch 2/100
18000/18000 [==============================] - 64s 4ms/step - loss: 481.7941 - reco_loss: 481.7941 - kl_loss: 3.2321e-07 - val_loss: 415.1098 - val_reco_loss: 415.1098 - val_kl_loss: 1.2368e-05 - lr: 0.0010
Epoch 3/100
18000/18000 [==============================] - 64s 4ms/step - loss: 485.2541 - reco_loss: 485.2541 - kl_loss: 1.9278e-08 - val_loss: 415.1019 - val_reco_loss: 415.1019 - val_kl_loss: 9.8944e-06 - lr: 0.0010
Epoch 4/100
18000/18000 [==============================] - 64s 4ms/step - loss: 485.2500 - reco_loss: 485.2500 - kl_loss: 1.8990e-08 - val_loss: 415.0997 - val_reco_loss: 415.0997 - val_kl_loss: 2.4736e-06 - lr: 0.0010
Epoch 5/100
17998/18000 [============================>.] - ETA: 0s - loss: 486.1215 - reco_loss: 486.1215 - kl_loss: 1.8948e-08
